# Random number transformations
In this notebook we will explore random number generators with bias. This was an idea that came to mind while learning about bernoulli trials.  
Let us assume that we have a PRNG that generates a single bit uniformly (either 0 or 1 with equal probability). Is it possible to create another PRNG using this one that returns a biased result? After tinkering with boolean algebra, I came to realize that it is beautifully simple.  
Let us call our PRNG that returns a bit RB. What will (RB AND RB) return? We know that the AND operation returns 1 only when both its operands are 1. This means that this operation will return 1 25% of the time. Similarly if we do and OR operation, we will get 0 only 25% of the time.

In [12]:
import random

""" 
Here we use the random library to implement our random bit generator. Since randint() uses the mersenne twister
the number returned will be uniformly distributed over 0 and 1.
"""    
def RandBit():
    return random.randint(0, 1)

In [15]:
# Using bitwise and to get 25% probability of a 1
def RandBit25():
    return RandBit() and RandBit()

# Using bitwise or to get 75% probability of a 1
def RandBit75():
    return RandBit() or RandBit()

check25 = 0
for i in range(0, 10000):
    if RandBit25() == 1:
        check25 += 1
        
check75 = 0
for i in range(0, 10000):
    if RandBit75() == 1:
        check75 += 1
        
print("Using bitwise and returned 1 " + str(check25/100) + "% of the time")
print("Using bitwise or returned 1 " + str(check75/100) + "% of the time")

Using bitwise and returned 1 25.67% of the time
Using bitwise or returned 1 75.28% of the time


As we can see above, the PRNGs returned 1 with the expected probability. Using the knowledge we gained above, can we generate 1 with any probability of our choice? After a little bit of thought, it is easy to see that this problem can be rephrased to make it easier to understand. Obtaining a 1 with probability x is equivalent to generating x equiprobable numbers. If the number so obtained is the xth number we return 1 otherwise we return 0. 
To generate x numbers, a simple thing to do would be to take x random bits and add them. However this constitutes a binomial distribution, and hence the probabilities so obtained will not be equal.  
Another possibility that only works for powers of 2 is to generate a bitstring of length log<sub>2</sub>(x) and convert it back to decimal. This will give us a uniformly distributed random variable over the range (0, x-1).  
For a number that isn't a power of 2, we can simply take the next greatest power of 2 and neglect the values that exceed x. This however is wasteful, and the runtime will be unbounded because it is possible to get an infinite sequence of numbers greater than the desired number. We also waste all the bits that we neglect which in the worst case can be half the bits we generate. To get past this hurdle, and generate uniformly distributed numbers we do the following :  
- Let n be the upper bound on the number ie we want to generate numbers that lie in [0, n-1]
- Let N be some large number such that the computer can represent 2N without any loss of data
- We keep 2 variables (static) so that we can carry over the 'lost data'.
- Let r = 0 and m = 1 (these two variables)
Keeping the above in mind we devise the following algorithm : 
* WHILE m < N DO r : = 2r + RandBit(); m = 2m; (r is a random variable of modulus m)
* Divide m by n : m = nq + b
*  IF r >= nq, let m := m - nq, r := r - nq (r is still a random variable of modulus m), and go to step 1.
* Otherwise, let x : = r mod n, r : = [r/n], and m : = q, and return x.
When we reach step 4, we return x which is a uniformly distributed random variable over [0, n-1] as r is uniformly distributed over nq. 

In [20]:
r = 0
m = 1
N = 4294967296

def UniformDistribution(n):
    global r
    global m
    global N
    
    while True:
        while m < N:
            r = 2*r + RandBit()
            m *= 2
        
        q = m//n
    
        if r >= n*q:
            m -= n*q
            r -= n*q
        else:
            m = q
            x = r%n
            r = r//n
            return x

In [23]:
prob = [0]*10
for i in range(0, 10000):
    prob[UniformDistribution(10)] += 1

for i in range(0, 10):
    prob[i] /= 10000
print(prob)

[0.0982, 0.0982, 0.1032, 0.1026, 0.0998, 0.1028, 0.0989, 0.1048, 0.0942, 0.0973]


As we can see, the numbers generated have a near uniform distribution. Using this fact, we can create any spread of random numbers with arbitrary probabilities from a simple random bit generator. Here my random bit generator uses an inbuilt PRNG, but it is easy (and fast) to extract a random bit. This can be used where we want an arbitrary spread of probabilities quickly and efficiently using only a random bit generator.